In [1]:
from surprise.dataset import Dataset

# загружаем встроенный набор данных
# если не загружен - попросит загрузить (это быстро)
data = Dataset.load_builtin(name="ml-1m")

In [2]:
from surprise.model_selection import train_test_split

# разбиваем на обучающую и валидационную выборки
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [3]:
%%time
from surprise import SVD

# обучаем модель
algo = SVD(random_state=42)
algo.fit(trainset)

CPU times: user 42.3 s, sys: 0 ns, total: 42.3 s
Wall time: 42.4 s


In [4]:
%%time
# получаем рекомендации на валидационном наборе
predictions = algo.test(testset)

CPU times: user 2.14 s, sys: 50 ms, total: 2.19 s
Wall time: 2.2 s


In [5]:
%%time
from surprise.evaluate import accuracy

# оцениваем качество
accuracy.rmse(predictions)

RMSE: 0.8729
CPU times: user 150 ms, sys: 10 ms, total: 160 ms
Wall time: 164 ms


In [6]:
# возьмём определённого пользователя
# и посмотрим на его оценки
user_id = 42
user_ratings = trainset.ur[user_id]
print("список оценок:", user_ratings)
num_ratings = len(user_ratings)
print("всего оценок:", num_ratings)
mean_rating = sum([rating for item_id, rating in user_ratings]) / num_ratings
print("средняя оценка:", mean_rating)

список оценок: [(41, 3.0), (363, 4.0), (603, 3.0), (481, 5.0), (271, 5.0), (686, 5.0), (1068, 4.0), (1097, 4.0), (440, 5.0), (1603, 4.0), (151, 5.0), (1628, 5.0), (550, 5.0), (574, 5.0), (131, 5.0), (1570, 5.0), (112, 4.0), (265, 5.0), (1308, 3.0), (438, 5.0), (1562, 5.0), (520, 5.0), (828, 5.0), (210, 4.0), (63, 4.0), (934, 4.0), (394, 4.0), (832, 5.0), (325, 5.0), (2239, 4.0), (909, 4.0), (75, 4.0), (121, 5.0), (408, 5.0), (646, 5.0), (15, 5.0), (908, 5.0), (412, 5.0), (350, 5.0), (27, 5.0), (190, 5.0), (156, 5.0), (528, 5.0), (655, 4.0), (254, 5.0)]
всего оценок: 45
средняя оценка: 4.5777777777777775


In [7]:
# посмотрим, как устроено предсказание
item_id = 41
# "внешнее API" - сюда подаются реальные ID пользователей и фильмов
print(algo.predict(
    uid=trainset.to_raw_uid(user_id),
    iid=trainset.to_raw_iid(item_id),
    r_ui=dict(trainset.ur[user_id])[item_id]
))
# внутренее API - сюда подаются внутренние ID
print(algo.estimate(
    u=user_id,
    i=item_id
))
# формула предсказания
print(
    # скалярное произведение скрытых факторов пользователя и фильма
    algo.pu[user_id].dot(algo.qi[item_id]) +
    # смещение фильма
    algo.bi[item_id] +
    # смещение пользователя
    algo.bu[user_id] +
    # общее смещение датасета
    trainset.global_mean
)

user: 288        item: 3398       r_ui = 3.00   est = 3.83   {'was_impossible': False}
3.8250928507915827
3.8250928507915822


$$\hat{r}_{ui}=\mu+b_i+b_u+p_u\cdot q_i,$$ где

$\mu$ - средняя оценка на обучающей выборке

$b_i$ - смещение объекта (~средняя оценка объекта)

$u_i$ - смещение пользователя (~средее по оценкам пользователя)

$p_u$ - вектор скрытых факторов пользователя (~профиль, неименоваенные предпочтения)

$q_u$ - вектор скрытых факторов объекта (~неименованные фичи объекта вроде жанров)

In [8]:
import numpy as np

# скрытые факторы не нормированы
print(np.linalg.norm(algo.pu[user_id]))
# скрытые факторы не распределены от 0 до 1
print(algo.pu.min())
print(algo.qi.min())
# или от -1 до 1
print(algo.pu.max())
print(algo.qi.max())
# среднее и стандартное отклонение
print(algo.qi.mean())
print(algo.pu.mean())
print(algo.qi.std())
print(algo.pu.std())

1.1107987856767139
-1.0696380325527513
-0.9803457306146609
1.0947878033759693
0.9315214141413002
-0.0006434778125401047
3.5125448487828127e-05
0.14139676994807773
0.13119236468642578


In [9]:
# смещение пользователя не равно его средней оценке

print(mean_rating)
print(algo.bu[user_id])
print(algo.bu[user_id] + trainset.global_mean)

4.5777777777777775
0.5629317460462425
4.144421610037135


$$L=\sum\limits_{r_{ui}}\left(\hat{r}_{ui}-r_{ui}\right)^2+\lambda_i\sum\limits_ib_i^2+\lambda_u\sum\limits_ub_u^2+\lambda_p\sum\limits_u\left\lVert p_u\right\rVert^2+\lambda_q\sum\limits_i\left\lVert q_i\right\rVert^2$$ где

$\lambda_.$ - коэффициенты регуляризации (могут быть одинаковыми)